In [25]:
from pymcdm.methods import TOPSIS
import pandas as pd
import numpy as np
from mealpy.swarm_based.PSO import OriginalPSO
from pymcdm_reidentify.methods import STFN
from pymcdm_reidentify.normalizations import FuzzyNormalization

In [26]:
pf = pd.read_csv("test_data/test_data_small.csv")
pf

,alternatives,cat1,cat2,cat3,cat4,cat5
0,a1,200.0,114.55,112.33,87.74,23.47
1,a2,44.6,25.83,25.73,15.17,6.51
2,a3,135.8,211.83,178.55,107.39,25.81
3,a4,47.5,69.37,68.32,34.82,7.55
4,a5,164.7,10.10,8.14,5.68,1.27
5,a6,88.3,147.93,146.97,85.60,25.27
6,a7,200.0,100.00,96.34,67.15,20.22
7,a8,333.6,60.60,56.16,45.96,14.20
8,a9,78.8,17.99,16.71,9.48,3.36
9,a10,156.5,82.80,71.94,53.70,15.05


In [27]:
matrix = [
    [200, 114.55, 112.33, 87.74, 23.47],
    [44.6, 25.83, 25.73, 15.17, 6.51],
    [135.8, 211.83, 178.55, 107.39, 25.81],
    [47.5, 69.37, 68.32, 34.82, 7.55],
    [164.7, 10.1 , 8.14, 5.68, 1.27],
    [88.3, 147.93, 146.97, 85.6, 25.27],
    [200, 100, 96.34, 67.15, 20.22],
    [333.6, 60.6, 56.16, 45.96, 14.2],
    [78.8, 17.99, 16.71, 9.48, 3.36],
    [156.5, 82.8, 71.94, 53.7, 15.05]
]

In [28]:
# matrix = pf.to_numpy()[:, 1:].astype(float)
matrix = np.array(matrix)
weights = np.array([0.15, 0.228, 0.222, 0.21, 0.19])
types = np.array([1, 1, 1, -1, -1])
matrix


array([[200.  , 114.55, 112.33,  87.74,  23.47],
       [ 44.6 ,  25.83,  25.73,  15.17,   6.51],
       [135.8 , 211.83, 178.55, 107.39,  25.81],
       [ 47.5 ,  69.37,  68.32,  34.82,   7.55],
       [164.7 ,  10.1 ,   8.14,   5.68,   1.27],
       [ 88.3 , 147.93, 146.97,  85.6 ,  25.27],
       [200.  , 100.  ,  96.34,  67.15,  20.22],
       [333.6 ,  60.6 ,  56.16,  45.96,  14.2 ],
       [ 78.8 ,  17.99,  16.71,   9.48,   3.36],
       [156.5 ,  82.8 ,  71.94,  53.7 ,  15.05]])

In [29]:
print("Weights:", weights)
print("Types:", types)

Weights: [0.15  0.228 0.222 0.21  0.19 ]
Types: [ 1  1  1 -1 -1]


In [30]:
topsis = TOPSIS()
expert_pref = topsis(matrix, weights, types)
print("pref =", np.round(expert_pref, 4))
expert_rank = topsis.rank(expert_pref)
print("rank =", expert_rank)


pref = [0.4129 0.4255 0.5165 0.4576 0.4677 0.4558 0.427  0.4645 0.4462 0.4181]
rank = [10.  8.  1.  4.  2.  5.  7.  3.  6.  9.]


In [31]:
def make_bounds(data_matrix):
    bounds = np.array([[np.min(data_matrix[:, i]), np.max(data_matrix[:, i])]
                      for i in range(data_matrix.shape[1])])
    return bounds

bounds = make_bounds(matrix)
bounds

array([[ 44.6 , 333.6 ],
       [ 10.1 , 211.83],
       [  8.14, 178.55],
       [  5.68, 107.39],
       [  1.27,  25.81]])

In [32]:
stoch = OriginalPSO(epoch=100, pop_size=50)
stfn = STFN(stoch.solve, TOPSIS(), bounds, weights)

In [33]:
stfn.fit(matrix, expert_rank)

2025/12/07 02:13:52 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.
2025/12/07 02:13:53 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 1, Current best: -0.33223140495867765, Global best: -0.33223140495867765, Runtime: 0.01689 seconds
2025/12/07 02:13:53 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 2, Current best: -0.33223140495867765, Global best: -0.33223140495867765, Runtime: 0.01778 seconds
2025/12/07 02:13:53 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 3, Current best: -0.33223140495867765, Global best: -0.33223140495867765, Runtime: 0.01687 seconds
2025/12/07 02:13:53 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 4, Current best: -0.33223140495867765, Global best: -0.33223140495867765, Runtime: 0.01595 seconds
2025/12/07 02:13:53 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 5, Current best: -0.33223140495867765, Global best:

In [34]:
print("Diagnostyka danych wejściowych:")
print("matrix shape:", matrix.shape)
print("matrix min/max:", np.min(matrix, axis=0), np.max(matrix, axis=0))
print("bounds:", bounds)
print("Czy bounds są szerokie:", (bounds[:,1] - bounds[:,0]))
print("weights:", weights)
print("expert_rank:", expert_rank)
print("Czy expert_rank są unikalne:", len(np.unique(expert_rank)), "z", len(expert_rank))
print("Czy są powtórzenia w expert_rank:", np.unique(expert_rank, return_counts=True))

Diagnostyka danych wejściowych:
matrix shape: (10, 5)
matrix min/max: [44.6  10.1   8.14  5.68  1.27] [333.6  211.83 178.55 107.39  25.81]
bounds: [[ 44.6  333.6 ]
 [ 10.1  211.83]
 [  8.14 178.55]
 [  5.68 107.39]
 [  1.27  25.81]]
Czy bounds są szerokie: [289.   201.73 170.41 101.71  24.54]
weights: [0.15  0.228 0.222 0.21  0.19 ]
expert_rank: [10.  8.  1.  4.  2.  5.  7.  3.  6.  9.]
Czy expert_rank są unikalne: 10 z 10
Czy są powtórzenia w expert_rank: (array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))


In [35]:
# Test funkcji celu STFN na losowych parametrach
import random
print("Test funkcji celu STFN (loss):")
for i in range(5):
    # Losowe parametry z zakresu bounds
    params = np.array([random.uniform(bounds[j,0], bounds[j,1]) for j in range(bounds.shape[0])])
    # Wywołanie funkcji celu (loss) bezpośrednio
    try:
        loss = stfn._loss(params, matrix, expert_rank)
        print(f"Próba {i+1}: loss = {loss}, params = {params}")
    except Exception as e:
        print(f"Próba {i+1}: Błąd: {e}")

Test funkcji celu STFN (loss):
Próba 1: Błąd: 'STFN' object has no attribute '_loss'
Próba 2: Błąd: 'STFN' object has no attribute '_loss'
Próba 3: Błąd: 'STFN' object has no attribute '_loss'
Próba 4: Błąd: 'STFN' object has no attribute '_loss'
Próba 5: Błąd: 'STFN' object has no attribute '_loss'


In [36]:
# Wylistuj wszystkie metody i atrybuty obiektu stfn, aby znaleźć funkcję celu
print("Atrybuty/metody obiektu stfn:")
for attr in dir(stfn):
    if not attr.startswith('__'):
        print(attr)

Atrybuty/metody obiektu stfn:
_SFN__x_train
_SFN__y_train
_StochasticIdentification__x_train
_StochasticIdentification__y_train
_abc_impl
base
cores
fit
fitness
get_fuzzy_numbers
lb
method
ub
weights


In [37]:
# Test funkcji celu: fitness
import random
print("Test funkcji celu STFN (fitness):")
for i in range(5):
    params = np.array([random.uniform(bounds[j,0], bounds[j,1]) for j in range(bounds.shape[0])])
    try:
        result = stfn.fitness(params, matrix, expert_rank)
        print(f"Próba {i+1}: fitness = {result}, params = {params}")
    except Exception as e:
        print(f"Próba {i+1}: Błąd: {e}")

Test funkcji celu STFN (fitness):
Próba 1: Błąd: SFN.fitness() takes from 2 to 3 positional arguments but 4 were given
Próba 2: Błąd: SFN.fitness() takes from 2 to 3 positional arguments but 4 were given
Próba 3: Błąd: SFN.fitness() takes from 2 to 3 positional arguments but 4 were given
Próba 4: Błąd: SFN.fitness() takes from 2 to 3 positional arguments but 4 were given
Próba 5: Błąd: SFN.fitness() takes from 2 to 3 positional arguments but 4 were given


In [38]:
print(f"STFN cores: {stfn.cores}")

STFN cores: [122.26356392 179.72410311  54.50424629  39.66871189  10.09170941]


In [39]:
def rw(rankx, ranky, n):
    suma = 0
    for i in range(n):
        suma += ((
            (rankx[i]-ranky[i])**2)
            *((n-rankx[i]+1)+(n-ranky[i]+1)
                    ))
    suma = 6 * suma
    denominator = n**4 + n**3 - n**2 - n
    if denominator == 0:
        return 0
    suma = suma / denominator
    return 1-suma

def WS(rankx, ranky, n):
    suma = 0
    for i in range(n):
        eq = 2 ** (-float(rankx[i]))
        eq2 = abs(rankx[i] - ranky[i]) / max(abs(1 - rankx[i]), abs(n - rankx[i]))
        suma += eq * eq2
    return 1 - suma

In [40]:
ob_norm = FuzzyNormalization(stfn())
body = TOPSIS(ob_norm)
new_types = np.ones(matrix.shape[1])
new_pref = body(matrix, weights, new_types)
new_rank = body.rank(new_pref)

In [41]:
print("new rank:", new_rank)
print("old rank:", expert_rank)
print("RW:", rw(expert_rank, new_rank, matrix.shape[0]))
print("WS:", WS(expert_rank, new_rank, matrix.shape[0]))

new rank: [ 5.  7.  8.  3.  9.  6.  4.  2. 10.  1.]
old rank: [10.  8.  1.  4.  2.  5.  7.  3.  6.  9.]
RW: -0.3024793388429752
WS: 0.3383773561507937
